In [1]:
import codecs,re
import pandas as pd
import itertools as it

In [2]:
trainingDataFile="/Users/gilles/Github/sigmorphon2016/data/german-task2-train"
cat="V"

In [3]:
with codecs.open(trainingDataFile,"r",encoding="utf8") as input:
    trainingData=input.readlines()

In [4]:
inflectedPairs=[]
for line in trainingData:
    (attributes1,form1,attributes2,form2)=line.strip().split("\t")
    (pos1,atts1)=attributes1.split(",",1)
    (pos2,atts2)=attributes2.split(",",1)
    inflectedPairs.append([(pos1,atts1,form1),(pos2,atts2,form2)])

In [5]:
def listAVs(chaine):
    dictAVs={}
    avs=chaine.split(",")
    for av in avs:
        attribute,value=av.split("=")
        m=re.match(ur"{(.+)\/(.+)}",value)
        if m:
            dictAVs[attribute]=set([m.group(1),m.group(2)])
        else:
            dictAVs[attribute]=set([value])
    return dictAVs

In [6]:
attributes={}
dimensions=set()
inflectedVerbs=[]
for c1,c2 in inflectedPairs:
    if cat in c1[0] and cat in c2[0]:
        inflectedVerbs.append((c1,c2))
        avs=listAVs(c1[1])
        for av in avs:
#            if "gen" in av: print c1, c2
            dimensions.add(av)
            if not av in attributes:
                attributes[av]=set()
            attributes[av]|=avs[av]
        avs=listAVs(c2[1])
        for av in avs:
#            if "gen" in av: print c2, c1
            dimensions.add(av)
            if not av in attributes:
                attributes[av]=set()
            attributes[av]|=avs[av]
repForme=[u"form"]+list(dimensions)

In [7]:
lstColumns=[a+"1" for a in repForme]+[a+"2" for a in repForme]
pdPaires=pd.DataFrame(columns=lstColumns)
pdFormes=pd.DataFrame(columns=repForme)
pdFormes

,form,mood,per,num,tense,aspect,finite


In [8]:
num=0
for c1,c2 in inflectedVerbs:
    avs=listAVs(c1[1])
    pdAVs={av:list(avs[av]) for av in avs}
    pdAVs[u"form"]=[c1[2]]
    produitAVs=list(it.product(*pdAVs.values()))
    for produitAV in produitAVs:
        dictLigne={a:[b] for (a,b) in zip(pdAVs.keys(),produitAV)}
        pdLigne=pd.DataFrame.from_dict(dictLigne)
        pdFormes=pd.concat([pdFormes,pdLigne],ignore_index=True)
    avs=listAVs(c2[1])
    pdAVs={av:list(avs[av]) for av in avs}
    pdAVs[u"form"]=[c2[2]]
    produitAVs=list(it.product(*pdAVs.values()))
    for produitAV in produitAVs:
        dictLigne={a:[b] for (a,b) in zip(pdAVs.keys(),produitAV)}
        pdLigne=pd.DataFrame.from_dict(dictLigne)
        pdFormes=pd.concat([pdFormes,pdLigne],ignore_index=True)
pdFormes

,aspect,finite,form,mood,num,per,tense
0,NaN,NFIN,aktualisieren,NaN,NaN,NaN,NaN
1,PFV,NaN,aktualisierten,SBJV,PL,3,PST
2,PFV,NaN,aktualisierten,COND,PL,3,PST
3,NaN,NFIN,bedanken,NaN,NaN,NaN,NaN
4,PFV,NaN,bedankten,SBJV,PL,3,PST
5,PFV,NaN,bedankten,COND,PL,3,PST
6,NaN,NFIN,beerdigen,NaN,NaN,NaN,NaN
7,PFV,NaN,beerdigte,SBJV,SG,3,PST
8,PFV,NaN,beerdigte,COND,SG,3,PST
9,NaN,NFIN,belohnen,NaN,NaN,NaN,NaN


In [9]:
listFormes=[]
for c1,c2 in inflectedVerbs:
    avs=listAVs(c1[1])
    pdAVs={av:avs[av] for av in avs}
    produitAVs=list(it.product(*pdAVs.values()))
    for produitAV in produitAVs:
        listFormes.append([c1[2],c1[2],"-".join(produitAV)])
    avs=listAVs(c2[1])
    pdAVs={av:avs[av] for av in avs}
    produitAVs=list(it.product(*pdAVs.values()))
    for produitAV in produitAVs:
        listFormes.append([c1[2],c2[2],"-".join(produitAV)])


In [10]:
pdLexique=pd.DataFrame(listFormes,columns=[u"clé","forme","case"])
pdLexique

,clé,forme,case
0,aktualisieren,aktualisieren,NFIN
1,aktualisieren,aktualisierten,PL-PST-SBJV-PFV-3
2,aktualisieren,aktualisierten,PL-PST-COND-PFV-3
3,bedanken,bedanken,NFIN
4,bedanken,bedankten,PL-PST-SBJV-PFV-3
5,bedanken,bedankten,PL-PST-COND-PFV-3
6,beerdigen,beerdigen,NFIN
7,beerdigen,beerdigte,SG-PST-SBJV-PFV-3
8,beerdigen,beerdigte,SG-PST-COND-PFV-3
9,belohnen,belohnen,NFIN


In [11]:
pdParadigmes=pdLexique.pivot_table(columns=['case'],index=u"clé",aggfunc=lambda x:x)

ValueError: Function does not reduce

In [ ]:
casesPaire=["NFIN","SG-PRS-IND-2"]
pdParadigmes["forme"][casesPaire].dropna()